In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import (
    Dense,
    Dropout,
    BatchNormalization,
    Input,
    LeakyReLU,
)
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import numpy as np
import os

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

# 加载MNIST数据集
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 数据预处理：归一化
x_train = x_train / 255.0
x_test = x_test / 255.0

x_train = x_train.reshape(-1, 28, 28, 1)  # 调整形状以匹配模型输入
x_test = x_test.reshape(-1, 28, 28, 1)

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# 数据增强
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    zoom_range=0.5,
    shear_range=0.5,
    fill_mode="nearest",
)
datagen.fit(x_train.reshape(-1, 28, 28, 1))  # 数据增强需要原始形状

# 构建多层感知机（MLP）神经网络模型
model = Sequential(
    [
        Input(shape=(28, 28, 1)),  # 输入层
        
        tf.keras.layers.Flatten(),  # 展平图像
        Dense(2048),  # 隐藏层 1
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),

        Dense(1024),  # 隐藏层 2
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),

        Dense(512),  # 隐藏层 3
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),

        Dense(256),  # 隐藏层 4
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),

        Dense(128),  # 隐藏层 5
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),

        Dense(64),  # 隐藏层 6
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),

        Dense(32),  # 隐藏层 7
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),

        Dense(10, activation="softmax"),  # 输出层
    ]
)

# 编译模型
model.compile(
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001),  # 使用RMSprop优化器
    loss="categorical_crossentropy",  # 多分类交叉熵损失
    metrics=["accuracy"],
)

# 回调函数
reduce_lr = ReduceLROnPlateau(
    monitor="val_loss", factor=0.5, patience=8, min_lr=1e-6, verbose=1
)

early_stopping = EarlyStopping(
    monitor="val_loss", patience=15, restore_best_weights=True, verbose=1
)

# 训练模型
history = model.fit(
    datagen.flow(x_train.reshape(-1, 28, 28, 1), y_train, batch_size=128),  # 调整批量大小并恢复原始形状
    epochs=200,  # 增加训练轮数
    validation_data=(x_test.reshape(-1, 28, 28, 1), y_test),  # 恢复原始形状
    callbacks=[reduce_lr, early_stopping],
)

# 评估模型
test_loss, test_acc = model.evaluate(x_test, y_test)  # 使用调整后的测试数据形状
print(f"Test accuracy: {test_acc}")

# 保存模型
model.save("mnist_model_mlp_v2.keras")